In [39]:
#learn more about pyterrier: https://pyterrier.readthedocs.io/en/latest/

Aufgabe: Bitte nutzt jetzt die Json Datei, die ihr durch Aufgabe 1 erhalten habt. Führt den kompletten Indexierungsvorgang mit PyTerrrier durch. Der Python Code der von euch gefordert wird, ist durch ??? markiert.
Stellt euch die folgenden Fragen:
* Welche Spalten aus dem Dataframe könnten als Id und welche als Text Feld benutzt werden?
* Welche Felder sollen indexiert werden?
* Was ist wichtig für die Indexierungspipeline?
* Zusatzaufgabe: Wie könnten beim Suchvorgang Publikationen gefiltert werden, deren Erscheinungsjahr nicht zwischen 2015 und 2020 ist? Passt dazu eure Definition der Suchmaschine an.


In [41]:
import pyterrier as pt
import pandas as pd
import pickle
import numpy as np

In [42]:
path = "/home/katja/2024-2025-Data_Lirbrarian_Katja_Gödde/Modul_2/literature.json"
data = pd.read_json(path)

In [43]:
data

,query,title,url,year,authors,abstract,source
0,artificial intelligence,Artificial Intelligence in Agriculture,https://www.ijtsrd.com/engineering/electrical-...,2021,"[Matthew N. O. Sadiku, Sarhan M. Musa, Abayomi...",Artificial Intelligence is one of the emerging...,bibsonomy
1,artificial intelligence,Artificial Intelligence Benefit and Risks,https://www.ijtsrd.com/computer-science/artifi...,2020,[Seeta M. Chauhan],this article demonstrate Disadvantage of artif...,bibsonomy
2,artificial intelligence,Artificial Intelligence in Power Station,https://www.ijtsrd.com/engineering/electrical-...,2019,"[P. Naveen, S. Nikitha, P. Sudeesh, V. Vaishnavi]",With increased competitiveness in power genera...,bibsonomy
3,artificial intelligence,Artificial Intelligence Based Training and Pla...,http://www.ijtsrd.com/computer-science/artific...,2018,"[Krishanu Deb, Pankaj Agrawal, Harish Nawale, ...",Training and placement cell in colleges is to ...,bibsonomy
4,artificial intelligence,The Significance of Artificial Intelligence in...,https://www.ijtsrd.com/computer-science/artifi...,2023,[Dr. Atul Kumar Mishra],"In an increasingly digitalized world, the util...",bibsonomy
...,...,...,...,...,...,...,...
4737,large language model,Phonologically Aware Neural Model for Named En...,https://www.semanticscholar.org/paper/f1a8ff55...,2016,"[Akash Bharadwaj, David R. Mortensen, Chris Dy...",Named Entity Recognition is a well established...,semantic_scholar
4738,large language model,Large-Scale Distributed Language Modeling,https://www.semanticscholar.org/paper/591080c3...,2007,"[Ahmad Emami, K. Papineni, Jeffrey Scott Soren...",A novel distributed language model that has no...,semantic_scholar
4739,large language model,BERT for Joint Intent Classification and Slot ...,https://www.semanticscholar.org/paper/476029ac...,2019,"[Qian Chen, Zhu Zhuo, Wen Wang]",Intent classification and slot filling are two...,semantic_scholar
4740,large language model,Language and Translation Model Adaptation usin...,https://www.semanticscholar.org/paper/b281a9d0...,2008,"[M. Snover, B. Dorr, R. Schwartz]","Traditionally, statistical machine translation...",semantic_scholar


In [44]:
#initialise pyterrier
if not pt.started():
    pt.init()

/tmp/ipykernel_4450/2275133231.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [45]:
#pyterrier exspects a docno field
#data['docno'] = data["url"]
#data['text'] = data["title"]
data.rename(columns={'url': 'docno', 'title': 'text'}, inplace=True)

#year field should be string to simplify indexing 
data['year'] = data['year'].astype('str')

In [46]:
data

,query,text,docno,year,authors,abstract,source
0,artificial intelligence,Artificial Intelligence in Agriculture,https://www.ijtsrd.com/engineering/electrical-...,2021,"[Matthew N. O. Sadiku, Sarhan M. Musa, Abayomi...",Artificial Intelligence is one of the emerging...,bibsonomy
1,artificial intelligence,Artificial Intelligence Benefit and Risks,https://www.ijtsrd.com/computer-science/artifi...,2020,[Seeta M. Chauhan],this article demonstrate Disadvantage of artif...,bibsonomy
2,artificial intelligence,Artificial Intelligence in Power Station,https://www.ijtsrd.com/engineering/electrical-...,2019,"[P. Naveen, S. Nikitha, P. Sudeesh, V. Vaishnavi]",With increased competitiveness in power genera...,bibsonomy
3,artificial intelligence,Artificial Intelligence Based Training and Pla...,http://www.ijtsrd.com/computer-science/artific...,2018,"[Krishanu Deb, Pankaj Agrawal, Harish Nawale, ...",Training and placement cell in colleges is to ...,bibsonomy
4,artificial intelligence,The Significance of Artificial Intelligence in...,https://www.ijtsrd.com/computer-science/artifi...,2023,[Dr. Atul Kumar Mishra],"In an increasingly digitalized world, the util...",bibsonomy
...,...,...,...,...,...,...,...
4737,large language model,Phonologically Aware Neural Model for Named En...,https://www.semanticscholar.org/paper/f1a8ff55...,2016,"[Akash Bharadwaj, David R. Mortensen, Chris Dy...",Named Entity Recognition is a well established...,semantic_scholar
4738,large language model,Large-Scale Distributed Language Modeling,https://www.semanticscholar.org/paper/591080c3...,2007,"[Ahmad Emami, K. Papineni, Jeffrey Scott Soren...",A novel distributed language model that has no...,semantic_scholar
4739,large language model,BERT for Joint Intent Classification and Slot ...,https://www.semanticscholar.org/paper/476029ac...,2019,"[Qian Chen, Zhu Zhuo, Wen Wang]",Intent classification and slot filling are two...,semantic_scholar
4740,large language model,Language and Translation Model Adaptation usin...,https://www.semanticscholar.org/paper/b281a9d0...,2008,"[M. Snover, B. Dorr, R. Schwartz]","Traditionally, statistical machine translation...",semantic_scholar


In [47]:
#transfrom dataframe into list of dictionaries
data_dict = data.to_dict(orient='records')
data_dict

[{'query': 'artificial intelligence',
  'text': 'Artificial Intelligence in Agriculture',
  'docno': 'https://www.ijtsrd.com/engineering/electrical-engineering/38513/artificial-intelligence-in-agriculture/matthew-n-o-sadiku',
  'year': '2021',
  'authors': ['Matthew N. O. Sadiku',
   'Sarhan M. Musa',
   'Abayomi Ajayi-Majebi'],
  'abstract': 'Artificial Intelligence is one of the emerging technologies in the field of agriculture which tries to simulate human reasoning in intelligent systems. It is making a revolution in agriculture by replacing inefficient traditional methods with more efficient AI based methods. AI is used in agriculture in various ways such as automation, robots, drones, soil and crop monitoring, and predictive analytics. This paper provides various applications of AI tools in agriculture. Matthew N. O. Sadiku | Sarhan M. Musa | Abayomi Ajayi-Majebi Ärtificial Intelligence in Agriculture" Published in International Journal of Trend in Scientific Research and Develop

In [66]:
index_folder_mult = "./ai_llm_index"

fields = ['docno', 'text', 'authors', 'year', 'title']

indexer_mult = pt.IterDictIndexer(index_folder_mult, meta={'docno': 1024, 'text': 4096, 'year' : 1024}, overwrite=True, fields=True, text_attrs=fields)
index_ref_mult = indexer_mult.index(data_dict, fields=fields)

print(f"path to our index: {index_ref_mult.toString()}")

ValueError: Use fields and text_attrs constructor kwargs

In [ ]:
#index_mult = ???
index_mult = pt.IndexFactory.of(index_ref_mult)

In [ ]:
print(index_mult.getCollectionStatistics().toString())

In [ ]:
search_engine_mult = pt.BatchRetrieve(index_mult, wmodel="TF_IDF")

In [ ]:
res = search_engine_mult.search("sentiment analysis")

In [ ]:
search_engine = pt.BatchRetrieve(index_mult, wmodel="TF_IDF", metadata=["docno", "year"]) 

rank_filter = pt.apply.generic(lambda res : res[res["rank"] < 10])

search_engine_year_filter = search_engine >> rank_filter

In [ ]:
search_engine_mult.search("sentiment analysis")